# Quantum Gram-Schmidt Process

Quantum Gram-Schmidt algorithm is used to generate a complete row basis by performing a quantum version of *adapting sampling* (TODO:#subtopic?)
- The advantage of the adapting sampling is as follows:
    - rows which have larger orthogonal part to the previously sampled row submatrix, will be sampled with a larger probability.
    - This ensures that the complete basis is non-singular.